# Final Project
### Import precomp libs

In [1]:
import spacy
nlp = spacy.load('en') # this should take some time like 10s to load
%load_ext line_profiler

In [2]:
#import the spacy english model
# "python -m spacy download en"   to download english models

import pickle 
import numpy as np
import scipy as sp
import pandas as pd
from collections import Counter
from sklearn.feature_extraction import DictVectorizer
from scipy.sparse import coo_matrix, hstack

In [6]:
# PROJECT structure: data directory with the two csvs of train,test and the ipython notbook in its own directory
# loading the data
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
print train.shape, test.shape


(404290, 6) (2345796, 3)


/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
test.loc[379205]['question2'] = None

In [ ]:
from nltk.stem.porter import PorterStemmer
def stem():
    porter = PorterStemmer()
    for w in l:
        porter.stem(w)
stem('What are the stories of Kohinoor (Koh-i-Noor) Diamond? men walked to the stores to buy shirts')

In [ ]:
def parse(sentence):
    bow = []
    sentence = sentence.decode('utf-8')
    p = nlp(sentence)
    for word in p:
        if word.is_stop: continue
        bow.append(word.lemma_)
    return Counter(bow)
        
parse('What are the stories of Kohinoor (Koh-i-Noor) Diamond? men walked to the stores to buy shirts')

In [ ]:
def print_vecs(corpus):
    vect_list = []
    results = []
    n = len(train)
    for i in range(n): 
        if i > 10: return 
        f = train.loc[i]
        q1, q2 = f['question1'], f['question2']
        first = parse(q1)
        first.subtract(parse(q2))
        print first
print_vecs(train)

In [ ]:
def make_vecs(corpus, results=False):
    vect_list = []
    results = []
    n = len(corpus)
    for i in range(n): 
        if i % 10000 == 0: print i
        f = corpus.loc[i]
        q1, q2 = f['question1'], f['question2']
        try: 
            p1 = parse(q1)
            p2 = parse(q2)
            diff1 = p1 - p2
            diff2 = p2 - p1
            vect_list.append((diff1, diff2))
            if results: results.append(f['is_duplicate'])
        except Exception as e:
            vect_list.append('')
            print(i, e)
    return fv, results

In [ ]:
fv_train, target = make_vecs(train, results=True)

In [ ]:
fv_test, results = make_vecs(test)

In [ ]:
# with open('fv_train','w') as f:
#     pickle.dump(fv_train_,f)
# with open('target','w') as f:
#     pickle.dump(target,f)
# with open('fv_test','w') as f:
#     pickle.dump(fv_test,f)

load from pickle

In [ ]:
fv_train_p = pickle.load(open('fv_train', 'rb'))
# fv_test_p = pickle.load(open('fv_test', 'rb'))
target_p = pickle.load(open('target', 'rb'))

In [ ]:
print(fv_train_p[10])

In [ ]:
def vectorize(vectors):
    v1, v2 = zip(*vectors)
    vectorizer = DictVectorizer()
    x1 = vectorizer.fit_transform(v1)
    x2 = vectorizer.fit_transform(v2)
    X = hstack([x1,x2])
    return X

In [ ]:
X = vectorize(fv_train_p)
target = target_p

In [ ]:
X.shape

#### Validation

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
clfs = [SVC(kernel='linear', probability=True)] 
#[MultinomialNB(), LogisticRegression()]

for clf in clfs:
    print clf
    scores = cross_val_score(clf, X, target, cv=3, scoring='neg_log_loss')
    print "Log Loss: %0.5f" % scores.mean()

### Testing

In [ ]:
f = open('submission', 'rb'))
f.write('test_id,is_duplicate')
fv_test = vectorize(fv_test_p)
clf = MultinomialNB().fit(X, target)
predict_proba(fv_test_p)

In [ ]:
#this procedure will be mapped to each row
def eval_sysim(row):
    q1_txt = nlp(unicode(row['question1']))
    q2_txt = nlp(unicode(row['question2']))

    # feature: similarity measure from built-in spacy
    #word net similarity? or hamming distance of the strings?
    sy_sim = q1_txt.similarity(q2_txt)

    #feature: 
    return sy_sim
    
#unit test(s)
utest = train.loc[1]
print eval_sysim(utest)
utest = train.loc[3]
print eval_sysim(utest)
utest = train.loc[9]
print eval_sysim(utest)

here we see the pitfall in relying on the similarity measure of the two doccuments as their scope differs(invest in shares vs invest in shares of **INDIA**). To extend this baseline model we will look at parts of speech and entity recognition to help parse scope.

We can however use the similarity as the first filter to remove completely unrelated questions
